In [ ]:
from tqdm import tqdm
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, date_format, to_date, create_map, lit, when, lower, regexp_replace, substring, concat_ws
from pyspark.sql import functions as F
from pyspark.sql import Window
from itertools import chain
import os
from dotenv import load_dotenv
load_dotenv()

from datetime import datetime
def log(msg):
    tqdm.write(f"[{datetime.now():%Y-%m-%d %H:%M:%S}] {msg}")

In [2]:
# 📌 Bibliotecas Padrão do Python
import logging
import warnings
import itertools
from datetime import datetime
from typing import Literal

# 📌 Manipulação de Dados
import pandas as pd
import numpy as np

# 📌 Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime



# 📌 Barras de Progresso
from tqdm import tqdm

# 📌 Configurações Globais para Melhor Exibição dos Dados
warnings.simplefilter(action="ignore", category=UserWarning)  # Ignorar avisos gerais do usuário
warnings.simplefilter(action="ignore", category=FutureWarning)  # Ignorar avisos de futuras mudanças

# Exibição de ponto flutuante sem notação científica
pd.options.display.float_format = "{:.2f}".format
# Configuração do número máximo de colunas e linhas exibidas
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 65)

# Configuração do backend de gráficos
pd.options.plotting.backend = "plotly"
pd.options.display.colheader_justify = "center"

In [ ]:
# Caminhos das camadas
silver_warehouse_path = "/home/pedromurta/projects/observatorio/caged/data/observatorio_caged/silver"
gold_warehouse_path   = "/home/pedromurta/projects/observatorio/caged/data/observatorio_caged/gold"

# Inicialização da SparkSession
spark = SparkSession.builder \
    .appName("Leitura Silver / Escrita Gold") \
    .config("spark.sql.catalog.silver", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.silver.type", "hadoop") \
    .config("spark.sql.catalog.silver.warehouse", silver_warehouse_path) \
    .config("spark.sql.catalog.gold", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.gold.type", "hadoop") \
    .config("spark.sql.catalog.gold.warehouse", gold_warehouse_path) \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.fraction", "0.6") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memoryOverhead", "2g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .getOrCreate()


your 131072x1 screen size is bogus. expect trouble
25/05/12 15:55:05 WARN Utils: Your hostname, GEGOVE-DT-09 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/12 15:55:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/05/12 15:55:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df_silver = spark.read.table("silver.default.caged_silver")

In [5]:
#df_silver.writeTo("gold.default.caged_gold").partitionedBy("competencia").createOrReplace()


In [6]:
df_silver.printSchema()

root
 |-- competencia: string (nullable = true)
 |-- saldo_movimentacao: integer (nullable = true)
 |-- secao: string (nullable = true)
 |-- sub_classe: string (nullable = true)
 |-- segmento: string (nullable = true)
 |-- codigo_cbo: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- etnia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- salario: double (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- uf_sigla: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- admissao: integer (nullable = true)
 |-- demissao: integer (nullable = true)



In [7]:
df_silver.show(5, truncate=False)

+-----------+------------------+-----+----------+--------+----------+---------------------------------------+-----+------+------+-------+--------------+--------+--------+--------+--------+
|competencia|saldo_movimentacao|secao|sub_classe|segmento|codigo_cbo|cbo                                    |idade|etnia |sexo  |salario|escolaridade  |uf_sigla|regiao  |admissao|demissao|
+-----------+------------------+-----+----------+--------+----------+---------------------------------------+-----+------+------+-------+--------------+--------+--------+--------+--------+
|2025-02    |-1                |M    |7112000   |OUTROS  |715315    |Armador de Estrutura de Concreto Armado|33   |Branca|Homem |2752.2 |Médio Completo|PR      |SUL     |0       |1       |
|2025-02    |1                 |G    |4781400   |OUTROS  |521110    |Vendedor de Comercio Varejista         |22   |Parda |Mulher|1518.0 |Médio Completo|BA      |NORDESTE|1       |0       |
|2025-02    |1                 |G    |4781400   |OUTROS

In [8]:
df_silver.groupBy("secao").count().orderBy("count", ascending=False).show()

+-----+--------+
|secao|   count|
+-----+--------+
|    G|52575028|
|    N|37464168|
|    C|32629433|
|    H|29807414|
|    F|21745906|
|    I|13508468|
|    A|11906780|
|    Q|10297371|
|    M| 7778761|
|    P| 6417101|
|    J| 4491390|
|    S| 4380751|
|    K| 2259192|
|    R| 1389364|
|    E| 1204671|
|    O| 1184350|
|    L|  822087|
|    B|  639000|
|    D|  206521|
|    T|   12186|
+-----+--------+
only showing top 20 rows



In [9]:
df_silver.count()

240725744

In [10]:
df_silver.groupBy("competencia").count().orderBy("competencia", ascending=True).show()

+-----------+------+
|competencia| count|
+-----------+------+
|    2010-01|123742|
|    2010-02|131238|
|    2010-03|163620|
|    2010-04|145002|
|    2010-05|151173|
|    2010-06|147699|
|    2010-07|148082|
|    2010-08|152758|
|    2010-09|152085|
|    2010-10|151479|
|    2010-11|152026|
|    2010-12|147305|
|    2011-01|152744|
|    2011-02|164507|
|    2011-03|174204|
|    2011-04|172312|
|    2011-05|188495|
|    2011-06|174201|
|    2011-07|160446|
|    2011-08|171137|
+-----------+------+
only showing top 20 rows



In [11]:
# Adiciona coluna de ano
df_silver = df_silver.withColumn("ano", substring("competencia", 1, 4))

In [36]:
df_silver = df_silver.withColumn(
    "faixa_etaria",
    when(col("idade") <= 17, "até 17")
    .when(col("idade").between(18, 24), "18-24")
    .when(col("idade").between(25, 29), "25-29")
    .when(col("idade").between(30, 39), "30-39")
    .when(col("idade").between(40, 49), "40-49")
    .when(col("idade").between(50, 64), "50-64")
    .otherwise("65+")
)


In [ ]:
# Filtra apenas motoristas
df_motoristas = df_silver.filter(col("codigo_cbo").cast("int").isin(782510))

# Mapeamento do salário mínimo por ano
df_motoristas = df_motoristas.withColumn("salario_minimo", 
    when(col("ano") == "2010", 510.00)
    .when(col("ano") == "2011", 545.00)
    .when(col("ano") == "2012", 622.00)
    .when(col("ano") == "2013", 678.00)
    .when(col("ano") == "2014", 724.00)
    .when(col("ano") == "2015", 788.00)
    .when(col("ano") == "2016", 880.00)
    .when(col("ano") == "2017", 937.00)
    .when(col("ano") == "2018", 954.00)
    .when(col("ano") == "2019", 998.00)
    .when(col("ano") == "2020", 1045.00)
    .when(col("ano") == "2021", 1100.00)
    .when(col("ano") == "2022", 1212.00)
    .when(col("ano") == "2023", 1320.00)
    .when(col("ano") == "2024", 1412.00)
    .when(col("ano") == "2025", 1518.00)
)

# Agregações principais
agregacoes = df_motoristas.groupBy("ano").agg(
    F.round(F.avg("salario"), 2).alias("salario_medio"),
    F.round(F.avg("salario") / F.first("salario_minimo"), 2).alias("salario_medio_sm"),
    F.round(F.avg("idade"), 2).alias("idade_media"),
    F.round(F.sum("admissao"), -2).alias("total_admissoes"),
    F.round(F.sum("demissao"), -2).alias("total_demissoes"),
    F.round(F.sum("admissao") - F.sum("demissao"), -2).alias("saldo_total")
)

# Escolaridade mais comum por ano
escolaridade_mais_comum = df_motoristas.groupBy("ano", "escolaridade").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("escolaridade").alias("escolaridade_mais_comum"))

# Etnia mais comum por ano
etnia_mais_comum = df_motoristas.groupBy("ano", "etnia").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("etnia").alias("etnia_mais_comum"))

# Sexo mais comum por ano
sexo_mais_comum = df_motoristas.groupBy("ano", "sexo").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("sexo").alias("sexo_mais_comum"))


# Unir tudo
resultado_final = agregacoes \
    .join(escolaridade_mais_comum, on="ano", how="left") \
    .join(etnia_mais_comum, on="ano", how="left") \
    .join(sexo_mais_comum, on="ano", how="left") \
    .orderBy("ano")

resultado_final.show(truncate=False)

25/05/13 15:25:41 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/13 15:25:41 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/13 15:25:41 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/13 15:25:42 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+----+-------------+----------------+-----------+---------------+---------------+-----------+-----------------------+----------------+---------------+-----------------------+
|ano |salario_medio|salario_medio_sm|idade_media|total_admissoes|total_demissoes|saldo_total|escolaridade_mais_comum|etnia_mais_comum|sexo_mais_comum|faixa_etaria_mais_comum|
+----+-------------+----------------+-----------+---------------+---------------+-----------+-----------------------+----------------+---------------+-----------------------+
|2010|1033.68      |2.03            |37.27      |179600         |157700         |21900      |Fundamental Completo   |Branca          |Homem          |30-39                  |
|2011|1132.92      |2.08            |37.35      |201100         |183900         |17200      |Médio Completo         |Branca          |Homem          |30-39                  |
|2012|1228.29      |1.97            |37.48      |204600         |191700         |12800      |Médio Completo         |Branca  

In [13]:
df_motoristas.count()

8869304

In [14]:
setor_h = df_silver.filter(col('secao') == 'H')

In [15]:
setor_h.count()

29807414

In [16]:
from pyspark.sql import functions as F

# Agrupar por competência e calcular total de admissões, demissões e saldo
resultados_por_mes = setor_h.groupBy("ano").agg(
   F.round(F.sum("admissao"), -2).alias("total_admissoes"),
   F.round(F.sum("demissao"), -2).alias("total_demissoes"),
   F.round(F.sum("admissao") - F.sum("demissao"), -2).alias("saldo_total")
).orderBy("ano")

# Exibir resultado
resultados_por_mes.show(truncate=False)

+----+---------------+---------------+-----------+
|ano |total_admissoes|total_demissoes|saldo_total|
+----+---------------+---------------+-----------+
|2010|955600         |810600         |145100     |
|2011|1071900        |934800         |137100     |
|2012|1057200        |987900         |69300      |
|2013|1110700        |1039700        |71000      |
|2014|1099100        |1057800        |41300      |
|2015|884700         |965800         |-81100     |
|2016|721000         |821500         |-100500    |
|2017|705600         |725500         |-19900     |
|2018|740500         |715200         |25400      |
|2019|805500         |783200         |22300      |
|2020|823800         |895700         |-71800     |
|2021|1086900        |975700         |111200     |
|2022|1201400        |1077800        |123600     |
|2023|1242600        |1131200        |111500     |
|2024|1388600        |1275300        |113400     |
|2025|377000         |337500         |39500      |
+----+---------------+---------

In [18]:
df_motoristas.groupBy("ano").agg(        
    F.round(F.sum("admissao"), -2).alias("total_admissoes"),
    F.round(F.sum("demissao"), -2).alias("total_demissoes"),
    F.round(F.sum("saldo_movimentacao"), -2).alias("saldo_total")
).show()

+----+---------------+---------------+-----------+
| ano|total_admissoes|total_demissoes|saldo_total|
+----+---------------+---------------+-----------+
|2025|         160700|         152000|       8700|
|2024|         622300|         600600|      21800|
|2023|         573400|         542500|      30900|
|2022|         540400|         480700|      59600|
|2021|         498500|         440300|      58200|
|2020|         391000|         366100|      24900|
|2013|         217400|         203300|      14000|
|2017|         138100|         135300|       2700|
|2014|         217100|         215800|       1400|
|2018|         136700|         120700|      15900|
|2015|         178300|         192400|     -14100|
|2010|         179600|         157700|      21900|
|2016|         147100|         168100|     -21000|
|2019|         164900|         147100|      17800|
|2011|         201100|         183900|      17200|
|2012|         204600|         191700|      12800|
+----+---------------+---------

In [ ]:
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import col, substring, when

# Adiciona coluna de ano
df_ano = df_silver.withColumn("ano", substring("competencia", 1, 4))

df_ano = df_silver.filter(col('secao') == 'H')

# Filtra apenas motoristas com o CBO 782510 (caminhoneiro)
df_motoristas = df_ano.filter(col("codigo_cbo").cast("int") == 782510)

# Mapeamento do salário mínimo por ano
df_motoristas = df_motoristas.withColumn("salario_minimo", 
    when(col("ano") == "2010", 510.00)
    .when(col("ano") == "2011", 545.00)
    .when(col("ano") == "2012", 622.00)
    .when(col("ano") == "2013", 678.00)
    .when(col("ano") == "2014", 724.00)
    .when(col("ano") == "2015", 788.00)
    .when(col("ano") == "2016", 880.00)
    .when(col("ano") == "2017", 937.00)
    .when(col("ano") == "2018", 954.00)
    .when(col("ano") == "2019", 998.00)
    .when(col("ano") == "2020", 1045.00)
    .when(col("ano") == "2021", 1100.00)
    .when(col("ano") == "2022", 1212.00)
    .when(col("ano") == "2023", 1320.00)
    .when(col("ano") == "2024", 1412.00)
    .when(col("ano") == "2025", 1518.00)
)

# Calcular proporção salário/salário mínimo para agregação segura
df_motoristas = df_motoristas.withColumn("salario_sm", col("salario") / col("salario_minimo"))

# Agregações principais
agregacoes = df_motoristas.groupBy("ano").agg(
    F.round(F.avg("salario"), 2).alias("salario_medio"),
    F.round(F.avg("salario_sm"), 2).alias("salario_medio_sm"),
    F.round(F.avg("idade"), 2).alias("idade_media"),
    F.round(F.sum("admissao"), -2).alias("total_admissoes"),
    F.round(F.sum("demissao"), -2).alias("total_demissoes"),
    F.round(F.sum("admissao") - F.sum("demissao"), -2).alias("saldo_total")
)

# Escolaridade mais comum por ano
escolaridade_mais_comum = df_motoristas.groupBy("ano", "escolaridade").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("escolaridade").alias("escolaridade_mais_comum"))

# Etnia mais comum por ano
etnia_mais_comum = df_motoristas.groupBy("ano", "etnia").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("etnia").alias("etnia_mais_comum"))

# Sexo mais comum por ano
sexo_mais_comum = df_motoristas.groupBy("ano", "sexo").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("sexo").alias("sexo_mais_comum"))

# Resultado final unificado
resultado_final = agregacoes \
    .join(escolaridade_mais_comum, on="ano", how="left") \
    .join(etnia_mais_comum, on="ano", how="left") \
    .join(sexo_mais_comum, on="ano", how="left") \
    .select(
        "ano", "salario_medio", "salario_medio_sm", "idade_media",
        "total_admissoes", "total_demissoes", "saldo_total",
        "escolaridade_mais_comum", "etnia_mais_comum", "sexo_mais_comum"
    ) \
    .orderBy("ano")

resultado_final.show(truncate=False)

25/05/12 16:00:27 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:00:27 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:00:27 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+----+-------------+----------------+-----------+---------------+---------------+-----------+-----------------------+----------------+---------------+
|ano |salario_medio|salario_medio_sm|idade_media|total_admissoes|total_demissoes|saldo_total|escolaridade_mais_comum|etnia_mais_comum|sexo_mais_comum|
+----+-------------+----------------+-----------+---------------+---------------+-----------+-----------------------+----------------+---------------+
|2010|1033.68      |2.03            |37.27      |179600         |157700         |21900      |Fundamental Completo   |Branca          |Homem          |
|2011|1132.92      |2.08            |37.35      |201100         |183900         |17200      |Médio Completo         |Branca          |Homem          |
|2012|1228.29      |1.97            |37.48      |204600         |191700         |12800      |Médio Completo         |Branca          |Homem          |
|2013|1340.21      |1.98            |37.75      |217400         |203300         |14000      |M

In [ ]:
#spark.sql('DROP TABLE IF EXISTS gold.default.caminhoneiros_saldo')

DataFrame[]

In [27]:
resultado_final.writeTo("gold.default.caminhoneiros_saldo").partitionedBy("ano").create()

25/05/12 16:23:48 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:23:48 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:23:48 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


In [24]:
# Lista de CBOs de motoristas
cbos_motoristas = [
    782310, 782320, 782405, 782410, 782415,
    82505, 782510, 782515, 7823, 7824, 7825, 7821, 7822
]

# Adiciona coluna de ano
df_ano = df_silver.withColumn("ano", substring("competencia", 1, 4))

# Filtra apenas motoristas
df_motoristas = df_ano.filter(col("codigo_cbo").cast("int").isin(cbos_motoristas))

# Mapeamento do salário mínimo por ano
df_motoristas = df_motoristas.withColumn("salario_minimo", 
    when(col("ano") == "2010", 510.00)
    .when(col("ano") == "2011", 545.00)
    .when(col("ano") == "2012", 622.00)
    .when(col("ano") == "2013", 678.00)
    .when(col("ano") == "2014", 724.00)
    .when(col("ano") == "2015", 788.00)
    .when(col("ano") == "2016", 880.00)
    .when(col("ano") == "2017", 937.00)
    .when(col("ano") == "2018", 954.00)
    .when(col("ano") == "2019", 998.00)
    .when(col("ano") == "2020", 1045.00)
    .when(col("ano") == "2021", 1100.00)
    .when(col("ano") == "2022", 1212.00)
    .when(col("ano") == "2023", 1320.00)
    .when(col("ano") == "2024", 1412.00)
    .when(col("ano") == "2025", 1518.00)
)

# Agregações principais
agregacoes = df_motoristas.groupBy("ano").agg(
    F.round(F.avg("salario"), 2).alias("salario_medio"),
    F.round(F.avg("salario") / F.first("salario_minimo"), 2).alias("salario_medio_sm"),
    F.round(F.avg("idade"), 2).alias("idade_media"),
    F.round(F.sum("admissao"), -2).alias("total_admissoes"),
    F.round(F.sum("demissao"), -2).alias("total_demissoes"),
    F.round(F.sum("admissao") - F.sum("demissao"), -2).alias("saldo_total")
)

# Escolaridade mais comum por ano
escolaridade_mais_comum = df_motoristas.groupBy("ano", "escolaridade").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("escolaridade").alias("escolaridade_mais_comum"))

# Etnia mais comum por ano
etnia_mais_comum = df_motoristas.groupBy("ano", "etnia").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("etnia").alias("etnia_mais_comum"))

# Sexo mais comum por ano
sexo_mais_comum = df_motoristas.groupBy("ano", "sexo").count() \
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("ano").orderBy(F.desc("count")))) \
    .filter("rn = 1").select("ano", col("sexo").alias("sexo_mais_comum"))

# Unir tudo
resultado_final_motoristas = agregacoes \
    .join(escolaridade_mais_comum, on="ano", how="left") \
    .join(etnia_mais_comum, on="ano", how="left") \
    .join(sexo_mais_comum, on="ano", how="left") \
    .orderBy("ano")

resultado_final_motoristas.show(truncate=False)

25/05/12 16:19:38 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:19:38 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:19:38 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+----+-------------+----------------+-----------+---------------+---------------+-----------+-----------------------+----------------+---------------+
|ano |salario_medio|salario_medio_sm|idade_media|total_admissoes|total_demissoes|saldo_total|escolaridade_mais_comum|etnia_mais_comum|sexo_mais_comum|
+----+-------------+----------------+-----------+---------------+---------------+-----------+-----------------------+----------------+---------------+
|2010|1056.26      |2.07            |37.5       |287100         |262000         |25100      |Fundamental Completo   |Branca          |Homem          |
|2011|1156.26      |2.12            |37.55      |322700         |298600         |24100      |Médio Completo         |Branca          |Homem          |
|2012|1251.46      |2.01            |37.69      |325500         |311700         |13800      |Médio Completo         |Branca          |Homem          |
|2013|1368.87      |2.02            |38.0       |347200         |332300         |14900      |M

In [28]:
resultado_final_motoristas.writeTo("gold.default.motoristas_saldo").partitionedBy("ano").create()

25/05/12 16:24:39 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:24:39 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/12 16:24:39 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


In [29]:
df_silver.show(5, truncate=False)

+-----------+------------------+-----+----------+--------+----------+---------------------------------------+-----+------+------+-------+--------------+--------+--------+--------+--------+----+
|competencia|saldo_movimentacao|secao|sub_classe|segmento|codigo_cbo|cbo                                    |idade|etnia |sexo  |salario|escolaridade  |uf_sigla|regiao  |admissao|demissao|ano |
+-----------+------------------+-----+----------+--------+----------+---------------------------------------+-----+------+------+-------+--------------+--------+--------+--------+--------+----+
|2025-02    |-1                |M    |7112000   |OUTROS  |715315    |Armador de Estrutura de Concreto Armado|33   |Branca|Homem |2752.2 |Médio Completo|PR      |SUL     |0       |1       |2025|
|2025-02    |1                 |G    |4781400   |OUTROS  |521110    |Vendedor de Comercio Varejista         |22   |Parda |Mulher|1518.0 |Médio Completo|BA      |NORDESTE|1       |0       |2025|
|2025-02    |1                

In [30]:
df_h = df_silver.filter(col('secao') == 'H')

# Filtra apenas caminhoneiros com o CBO 782510 (caminhoneiro)
df_caminhoneiros = df_h.filter(col("codigo_cbo").cast("int") == 782510)

df_caminhoneiros.count()

6225089

In [33]:
df_caminhoneiros.writeTo("gold.default.caminhoneiros_setor_h").partitionedBy("competencia").createOrReplace()

In [35]:
setor_h.count()

29807414

In [40]:
resultado_final_motoristas.count()

25/05/13 15:36:52 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/13 15:36:52 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression
25/05/13 15:36:52 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


16

In [42]:
# Lista de CBOs de motoristas
cbos_motoristas = [
    782310, 782320, 782405, 782410, 782415,
    82505, 782510, 782515, 7823, 7824, 7825, 7821, 7822
]

# Adiciona coluna de ano
df_ano = df_silver.withColumn("ano", substring("competencia", 1, 4))

# Filtra apenas motoristas
df_motoristas = df_ano.filter(col("codigo_cbo").cast("int").isin(cbos_motoristas))

df_motoristas.count()

13119533